From https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [24]:
from enum import Enum
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

# Working with data

In [25]:
class DataType(Enum):
    TRAIN = 1
    TEST = 2

def download_data(data_type: DataType) -> Dataset:
    if data_type == DataType.TRAIN:
        return datasets.FashionMNIST(
            root="data",
            train=True,
            download=True,
            transform=ToTensor()
        )
    elif data_type == DataType.TEST:
        return datasets.FashionMNIST(
            root="data",
            train=False,
            download=True,
            transform=ToTensor()
        )
    else:
        raise TypeError("Invalid data type")

In [26]:
training_data = download_data(DataType.TRAIN)
test_data = download_data(DataType.TEST)

In [27]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [28]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y [N, C, H, W]: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y [N, C, H, W]: torch.Size([64]) torch.int64


# Creating models

In [29]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [30]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [31]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# Optimizing the model parameters

In [32]:
# To train a model, we need a loss function and an optimizer.
loss_fn = nn.CrossEntropyLoss()
# SGD is stochastic gradient descent
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [33]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


In [34]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [35]:
epochs = 5
for t in range(epochs):
    print(f"EPOCH {t+1}\n---------------------")
    train(test_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Done!")

EPOCH 1
---------------------
loss: 2.300116 [   64/10000]
loss: 2.287999 [ 6464/10000]
Test Error: 
 Accuracy: 23.7%, Avg loss: 2.279402 

EPOCH 2
---------------------
loss: 2.276294 [   64/10000]
loss: 2.262526 [ 6464/10000]
Test Error: 
 Accuracy: 34.0%, Avg loss: 2.254862 

EPOCH 3
---------------------
loss: 2.252726 [   64/10000]
loss: 2.237109 [ 6464/10000]
Test Error: 
 Accuracy: 42.2%, Avg loss: 2.229859 

EPOCH 4
---------------------
loss: 2.228648 [   64/10000]
loss: 2.210715 [ 6464/10000]
Test Error: 
 Accuracy: 46.7%, Avg loss: 2.203369 

EPOCH 5
---------------------
loss: 2.203078 [   64/10000]
loss: 2.182374 [ 6464/10000]
Test Error: 
 Accuracy: 49.5%, Avg loss: 2.174452 

Done!


# Saving models

In [36]:
model_path = "model-data/model.pth"

torch.save(model.state_dict(), model_path)
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


# Loading models

In [37]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [38]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: {predicted}, Actual: {actual}")

Predicted: Ankle boot, Actual: Ankle boot
